In [ ]:
import os
import jsonref
from dotenv import dotenv_values
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiAnonymousAuthDetails, OpenApiTool, ToolSet, FileSearchTool, FilePurpose
from azure.identity import DefaultAzureCredential

In [ ]:
env = dotenv_values(".env")

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=env["FOUNDRY_CONN"]
)

Note... before you run the cell below, be sure you've deployed the API code to a public endpoint, and update the `servers` section in [api.json](../api/api.json) with the endpoint URL. This is how the agent knows where to send requests.

In [ ]:
with open('../api/api.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="weather_tool", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth)

In [ ]:
ny_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/new_york.txt', purpose=FilePurpose.AGENTS)
london_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/london.txt', purpose=FilePurpose.AGENTS)
tokyo_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/tokyo.txt', purpose=FilePurpose.AGENTS)

vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[ny_file.id, london_file.id, tokyo_file.id], name="city_info_vector_store")

file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

In [ ]:
toolset = ToolSet()
toolset.add(openapi)
toolset.add(file_search_tool)

agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="tourist-agent-2",
    instructions="You are a tourist info bot. Use only the provided tools and resources to answer user questions.",
    tools=toolset.definitions,
    tool_resources=toolset.resources,
)

In [ ]:
thread = project_client.agents.create_thread()

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Suggest a good tourist activity in London, and let me know if I'll need a jacket.",
)


In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")